In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, DotProduct, ExpSineSquared, Matern, RBF, RationalQuadratic, WhiteKernel

In [ ]:
file_path = "./data/samples/your_samples_file.csv"
data = pd.read_csv(file_path)

In [ ]:
data = data.drop(columns=['learned_kernel'])

In [ ]:
metrics = ['mse', 'lml', 'std', 'R2_tr', 'R2', 'R2_test']

In [ ]:
weight_R2_tr = 1.0
weight_R2 = 1.0
weight_R2_test = 1.0

In [ ]:
data['R2_score'] = (
    weight_R2_tr * data['R2_tr'] + 
    weight_R2 * data['R2'] + 
    weight_R2_test * data['R2_test']
) / (weight_R2_tr + weight_R2 + weight_R2_test)

In [ ]:
data['best_combination'] = (data['R2_score'] == data['R2_score'].max()).astype(int)

In [ ]:
X = data[metrics].values
y = data['best_combination'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Input(shape=(len(metrics),)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_split=0.2)

In [ ]:
y_pred = model.predict(X)

In [ ]:
best_index = y_pred.argmax()
best_kernel = data.iloc[best_index]

In [ ]:
print('Best kernel combination: ')
print(best_kernel)
print('Best Kernel: ', best_kernel[1])

In [ ]:
kernel = eval(best_kernel[0])

In [ ]:
file_name = "data/selected_kernel/your_kernel_file.txt"
with open(file_name, "w") as file:
    file.write(str(kernel))

print(f"Data saved in: {file_name}")